In [16]:
import numpy as np 
import pandas as pd

import sklearn as sk  
import seaborn as sns

import matplotlib.pyplot as plt

import opendatasets as od
import matplotlib.dates as mdates
import scipy.stats as stats

from scipy.stats import genpareto, norm, gaussian_kde

from scipy.stats import anderson, levene, bartlett, gumbel_r, genextreme
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

import ruptures as rpt 

In [2]:
#df = pd.read_csv('insurance/Unemployment_Insurance_Claims.csv')

In [17]:
df= pd.read_excel('insurance/foi-data-31Oct19.xlsx',sheet_name='Settled Claims')

In [18]:
df.head()
df.columns

Index(['Claim Reference', 'Policy Year', 'Policy Description',
       'Directorate Description', 'Department Name', 'Incident Date',
       'Notification Date', 'Status Flag', 'Payment Year', 'Payment Date',
       'Payment Type', ' Amount', 'Cause Description', 'Injury Description',
       'Damage Description'],
      dtype='object')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3605 entries, 0 to 3604
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Claim Reference          3605 non-null   object        
 1   Policy Year              3605 non-null   int64         
 2   Policy Description       3605 non-null   object        
 3   Directorate Description  3605 non-null   object        
 4   Department Name          3605 non-null   object        
 5   Incident Date            3605 non-null   datetime64[ns]
 6   Notification Date        3605 non-null   datetime64[ns]
 7   Status Flag              3605 non-null   object        
 8   Payment Year             3605 non-null   int64         
 9   Payment Date             3605 non-null   datetime64[ns]
 10  Payment Type             3605 non-null   object        
 11   Amount                  3605 non-null   float64       
 12  Cause Description        3605 non-

In [20]:
fig = go.Figure()

####Añadir la línea de Reclamaciones
fig.add_trace(go.Scatter(x=df['Payment Date'], y=df[' Amount'], mode='lines', name='No. the Reclamaciones'))

###Configurar el formato del eje x para las fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=20  # ajusta este valor según la cantidad de etiquetas de fecha que quieras mostrar
)

###Añadir títulos y etiquetas
fig.update_layout(
    title='Gráfico de Serie de Tiempo',
    xaxis_title='Fecha',
    yaxis_title='Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)
fig.show()

In [21]:
# Número de divisiones que quieres
n = 15  # Puedes ajustar este valor

# Definir el tamaño de cada segmento
segment_size = len(df) // n

# Crear una lista para almacenar los subconjuntos en orden descendente
subsets = []

# Crear los subconjuntos en orden descendente y almacenarlos en la lista
for i in range(n):
    start_idx = i * segment_size
    if i < n - 1:
        end_idx = (i + 1) * segment_size
    else:
        end_idx = len(df)  # Asegura que el último segmento llegue hasta el final
    
    # Crear subconjunto y añadirlo al principio de la lista (para orden descendente)
    subset = df.iloc[start_idx:end_idx]
    subsets.insert(0, subset)  # Usamos insert(0, ...) para que el subconjunto más reciente esté al inicio

# Crear la figura
fig = go.Figure()

# Añadir trazos para cada subconjunto, con numeración descendente
for i, subset in enumerate(subsets):
    # Añadir la línea del subconjunto correspondiente, con numeración descendente
    fig.add_trace(go.Scatter(x=subset['Payment Date'], 
                             y=subset[' Amount'], 
                             mode='lines', 
                             name=f'Segmento {i+1}'))  # La numeración ahora es ascendente en el orden visual

# Configurar el formato del eje x para las fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=20  # ajusta según la cantidad de etiquetas de fecha que quieras mostrar
)

# Añadir títulos y etiquetas
fig.update_layout(
    title='Gráfico de Serie de Tiempo con Segmentos en Orden Descendente',
    xaxis_title='Fecha',
    yaxis_title='Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

# Mostrar el gráfico
fig.show()

In [22]:
import plotly.express as px

# Elegir un subset cualquiera, por ejemplo, el segundo subset
subset = subsets[0]  # Puedes cambiar el índice a cualquier subset que desees

# Crear el gráfico de dispersión
fig = px.scatter(subset, x='Payment Date', y=' Amount', title='Gráfico de Dispersión del Subset')

# Mostrar el gráfico
fig.show()

In [23]:
# Definir el número de intervalos que quieres en el subset
n_intervals = 1  # Cambia este valor según lo que necesites

# Definir el tamaño de cada intervalo
interval_size = len(subset) // n_intervals

# Crear una lista para almacenar los subintervalos
subintervals = []

# Crear los subintervalos y guardarlos en la lista
for i in range(n_intervals):
    start_idx = i * interval_size
    if i < n_intervals - 1:
        end_idx = (i + 1) * interval_size
    else:
        end_idx = len(subset)  # Asegura que el último intervalo incluya el resto del subset
    
    # Crear el subintervalo
    subinterval = subset.iloc[start_idx:end_idx]
    subintervals.append(subinterval)

# Ahora, vamos a calcular las diferencias y graficarlas en una sola gráfica
fig = go.Figure()

# Añadir trazos para las diferencias de cada subintervalo
for i, subinterval in enumerate(subintervals):
    # Calcular las diferencias consecutivas
    diff_values = subinterval[' Amount'].diff().dropna()
    
    # Añadir el trazo para las diferencias en el subintervalo
    fig.add_trace(go.Scatter(x=subinterval['Payment Date'][1:], y=diff_values,
                             mode='lines', name=f'Diferencias Subintervalo {i+1}'))

# Configurar el formato del eje x para las fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10  # Ajusta según la cantidad de etiquetas de fecha que quieras mostrar
)

# Añadir títulos y etiquetas
fig.update_layout(
    title='Gráfico de Diferencias de Ingresos - Gastos',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

# Mostrar el gráfico
fig.show()

In [ ]:
# Definir el número de intervalos que quieres en el subset
n_intervals = 1  # Cambia este valor según lo que necesites

# Definir el tamaño de cada intervalo
interval_size = len(subset) // n_intervals

# Crear una lista para almacenar los subintervalos
subintervals = []

# Crear los subintervalos y guardarlos en la lista
for i in range(n_intervals):
    start_idx = i * interval_size
    if i < n_intervals - 1:
        end_idx = (i + 1) * interval_size
    else:
        end_idx = len(subset)  # Asegura que el último intervalo incluya el resto del subset
    
    # Crear el subintervalo
    subinterval = subset.iloc[start_idx:end_idx]
    subintervals.append(subinterval)

# Ahora, vamos a calcular las diferencias y graficarlas en una sola gráfica
fig = go.Figure()

# Añadir trazos para las diferencias de cada subintervalo
for i, subinterval in enumerate(subintervals):
    # Calcular las diferencias consecutivas
    diff_values = subinterval[' Amount'].diff().dropna()
    
    # Añadir el trazo para las diferencias en el subintervalo (modo dispersión)
    fig.add_trace(go.Scatter(
        x=subinterval['Payment Date'][1:], 
        y=diff_values,
        mode='markers',  # <-- Cambio aquí
        name=f'Diferencias Subintervalo {i+1}'
    ))

# Configurar el formato del eje x para las fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10
)

# Añadir títulos y etiquetas
fig.update_layout(
    title='Gráfico de Diferencias de Ingresos - Gastos',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

# Mostrar el gráfico
fig.show()

In [28]:
# Definir el número de intervalos que quieres en el subset
n_intervals = 1  # Cambia este valor según lo que necesites

# Definir el tamaño de cada intervalo
interval_size = len(subset) // n_intervals

# Crear una lista para almacenar los subintervalos
subintervals = []

# Crear los subintervalos y guardarlos en la lista
for i in range(n_intervals):
    start_idx = i * interval_size
    if i < n_intervals - 1:
        end_idx = (i + 1) * interval_size
    else:
        end_idx = len(subset)
    
    subinterval = subset.iloc[start_idx:end_idx]
    subintervals.append(subinterval)

# Crear figura
fig = go.Figure()

# Añadir los puntos para cada subintervalo
for i, subinterval in enumerate(subintervals):
    diff_values = subinterval[' Amount'].diff().dropna()
    dates = subinterval['Payment Date'][1:]
    
    # Filtrar valores positivos y negativos
    positive_mask = diff_values > 0
    negative_mask = diff_values < 0

    # Agregar puntos positivos
    fig.add_trace(go.Scatter(
        x=dates[positive_mask],
        y=diff_values[positive_mask],
        mode='markers',
        name=f'Ganancia (Positivos) Subintervalo de tiempo {i+1}',
        marker=dict(color='green', size=6, symbol='circle')
    ))

    # Agregar puntos negativos
    fig.add_trace(go.Scatter(
        x=dates[negative_mask],
        y=diff_values[negative_mask],
        mode='markers',
        name=f'Gastos/Perdidas (Negativas) subintervalo de tiempo{i+1}',
        marker=dict(color='red', size=6, symbol='circle')
    ))

# Añadir línea en y=0
fig.add_shape(
    type='line',
    x0=subset['Payment Date'].min(),
    x1=subset['Payment Date'].max(),
    y0=0,
    y1=0,
    line=dict(color='black', width=2, dash='dash'),
    xref='x',
    yref='y'
)

# Configurar el eje x
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10
)

# Etiquetas y título
fig.update_layout(
    title='Gráfico de Diferencias de Ingresos - Gastos',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

# Mostrar gráfico
fig.show()

In [35]:
import numpy as np

# Definir el número de intervalos que quieres en el subset
n_intervals = 1  # Cambia este valor según lo que necesites

# Definir el tamaño de cada intervalo
interval_size = len(subset) // n_intervals

# Crear una lista para almacenar los subintervalos
subintervals = []

# Crear los subintervalos y guardarlos en la lista
for i in range(n_intervals):
    start_idx = i * interval_size
    if i < n_intervals - 1:
        end_idx = (i + 1) * interval_size
    else:
        end_idx = len(subset)
    
    subinterval = subset.iloc[start_idx:end_idx]
    subintervals.append(subinterval)

# Crear figura
fig = go.Figure()

# Acumulador para todos los valores de diferencias (para percentiles)
all_diff_values = []

# Añadir los puntos para cada subintervalo
for i, subinterval in enumerate(subintervals):
    diff_values = subinterval[' Amount'].diff().dropna()
    dates = subinterval['Payment Date'][1:]
    
    all_diff_values.extend(diff_values)  # Guardar para percentiles
    
    # Filtrar valores positivos y negativos
    positive_mask = diff_values > 0
    negative_mask = diff_values < 0

    # Agregar puntos positivos
    fig.add_trace(go.Scatter(
        x=dates[positive_mask],
        y=diff_values[positive_mask],
        mode='markers',
        name=f'Ganancia (Positivos) Subintervalo {i+1}',
        marker=dict(color='green', size=6, symbol='circle')
    ))

    # Agregar puntos negativos
    fig.add_trace(go.Scatter(
        x=dates[negative_mask],
        y=diff_values[negative_mask],
        mode='markers',
        name=f'Gastos (Negativos) Subintervalo {i+1}',
        marker=dict(color='red', size=6, symbol='circle')
    ))

# Calcular percentiles 10 y 90
percentil_10 = np.percentile(all_diff_values, 10)
percentil_90 = np.percentile(all_diff_values, 90)

# Líneas horizontales para percentiles
fig.add_shape(
    type='line',
    x0=subset['Payment Date'].min(),
    x1=subset['Payment Date'].max(),
    y0=percentil_10,
    y1=percentil_10,
    line=dict(color='blue', width=2, dash='dot'),
    name='Percentil 10',
    xref='x',
    yref='y'
)

fig.add_shape(
    type='line',
    x0=subset['Payment Date'].min(),
    x1=subset['Payment Date'].max(),
    y0=percentil_90,
    y1=percentil_90,
    line=dict(color='orange', width=2, dash='dot'),
    name='Percentil 90',
    xref='x',
    yref='y'
)

# Etiquetas para los percentiles en anotaciones
fig.add_annotation(x=subset['Payment Date'].min(), y=percentil_10,
                   text=f'P10: {percentil_10:.2f}', showarrow=False,
                   yshift=-10, font=dict(color='blue'))

fig.add_annotation(x=subset['Payment Date'].min(), y=percentil_90,
                   text=f'P90: {percentil_90:.2f}', showarrow=False,
                   yshift=10, font=dict(color='orange'))

# Línea en y=0
fig.add_shape(
    type='line',
    x0=subset['Payment Date'].min(),
    x1=subset['Payment Date'].max(),
    y0=0,
    y1=0,
    line=dict(color='black', width=2, dash='dash'),
    xref='x',
    yref='y'
)

# Eje x
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10
)

# Layout
fig.update_layout(
    title='Gráfico de Diferencias de Ingresos - Gastos con Percentiles',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

fig.show()

In [37]:
import numpy as np

n_intervals = 1
interval_size = len(subset) // n_intervals
subintervals = []

for i in range(n_intervals):
    start_idx = i * interval_size
    if i < n_intervals - 1:
        end_idx = (i + 1) * interval_size
    else:
        end_idx = len(subset)
    subinterval = subset.iloc[start_idx:end_idx]
    subintervals.append(subinterval)

fig = go.Figure()

all_diff_values = []

for i, subinterval in enumerate(subintervals):
    diff_values = subinterval[' Amount'].diff().dropna()
    dates = subinterval['Payment Date'][1:]
    
    all_diff_values.extend(diff_values)

    # Calcular percentiles una vez que tengamos todos los valores
percentil_10 = np.percentile(all_diff_values, 10)
percentil_90 = np.percentile(all_diff_values, 90)

for i, subinterval in enumerate(subintervals):
    diff_values = subinterval[' Amount'].diff().dropna()
    dates = subinterval['Payment Date'][1:]

    # Máscaras
    positive = (diff_values > 0) & (diff_values <= percentil_90)
    negative = (diff_values < 0) & (diff_values >= percentil_10)
    extreme_high = diff_values > percentil_90
    extreme_low = diff_values < percentil_10

    # Puntos positivos normales
    fig.add_trace(go.Scatter(
        x=dates[positive],
        y=diff_values[positive],
        mode='markers',
        name=f'Ganancias normales {i+1}',
        marker=dict(color='green', size=6)
    ))

    # Puntos negativos normales
    fig.add_trace(go.Scatter(
        x=dates[negative],
        y=diff_values[negative],
        mode='markers',
        name=f'Pérdidas normales {i+1}',
        marker=dict(color='red', size=6)
    ))

    # Puntos extremadamente altos
    fig.add_trace(go.Scatter(
        x=dates[extreme_high],
        y=diff_values[extreme_high],
        mode='markers',
        name=f'Extremos de Ganancias {i+1}',
        marker=dict(color='gold', size=8, symbol='diamond')
    ))

    # Puntos extremadamente bajos
    fig.add_trace(go.Scatter(
        x=dates[extreme_low],
        y=diff_values[extreme_low],
        mode='markers',
        name=f'Extremos de Gastos {i+1}',
        marker=dict(color='purple', size=8, symbol='diamond')
    ))

# Línea y=0
fig.add_shape(
    type='line',
    x0=subset['Payment Date'].min(),
    x1=subset['Payment Date'].max(),
    y0=0,
    y1=0,
    line=dict(color='black', width=2, dash='dash'),
    xref='x',
    yref='y'
)

# Líneas de percentiles
fig.add_shape(
    type='line',
    x0=subset['Payment Date'].min(),
    x1=subset['Payment Date'].max(),
    y0=percentil_10,
    y1=percentil_10,
    line=dict(color='blue', width=1.5, dash='dot')
)

fig.add_shape(
    type='line',
    x0=subset['Payment Date'].min(),
    x1=subset['Payment Date'].max(),
    y0=percentil_90,
    y1=percentil_90,
    line=dict(color='orange', width=1.5, dash='dot')
)

fig.add_annotation(x=subset['Payment Date'].min(), y=percentil_10,
                   text=f'P10: {percentil_10:.2f}', showarrow=False,
                   yshift=-10, font=dict(color='blue'))

fig.add_annotation(x=subset['Payment Date'].min(), y=percentil_90,
                   text=f'P90: {percentil_90:.2f}', showarrow=False,
                   yshift=10, font=dict(color='orange'))

fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10
)

fig.update_layout(
    title='Diferencias Ingresos-Gastos con valores extremos resaltados',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

fig.show()

In [ ]:
# Definir el número de intervalos que quieres en el subset
n_intervals = 1  # Cambia este valor según lo que necesites

# Definir el tamaño de cada intervalo
interval_size = len(subset) // n_intervals

# Crear una lista para almacenar los subintervalos
subintervals = []

# Crear los subintervalos y guardarlos en la lista
for i in range(n_intervals):
    start_idx = i * interval_size
    if i < n_intervals - 1:
        end_idx = (i + 1) * interval_size
    else:
        end_idx = len(subset)
    
    subinterval = subset.iloc[start_idx:end_idx]
    subintervals.append(subinterval)

# Crear listas para las fechas y valores separados por signo
dates_positive = []
values_positive = []
dates_negative = []
values_negative = []

# Agrupar todos los datos positivos y negativos
for subinterval in subintervals:
    diff_values = subinterval[' Amount'].diff().dropna()
    dates = subinterval['Payment Date'][1:]
    
    for date, value in zip(dates, diff_values):
        if value > 0:
            dates_positive.append(date)
            values_positive.append(value)
        elif value < 0:
            dates_negative.append(date)
            values_negative.append(value)

# Crear la figura
fig = go.Figure()

# Agregar barras positivas (verdes)
fig.add_trace(go.Bar(
    x=dates_positive,
    y=values_positive,
    name='Diferencias Positivas',
    marker_color='green'
))

# Agregar barras negativas (rojas)
fig.add_trace(go.Bar(
    x=dates_negative,
    y=values_negative,
    name='Diferencias Negativas',
    marker_color='red'
))

# Añadir línea horizontal en y = 0
fig.add_shape(
    type='line',
    x0=subset['Payment Date'].min(),
    x1=subset['Payment Date'].max(),
    y0=0,
    y1=0,
    line=dict(color='black', width=2, dash='dash'),
    xref='x',
    yref='y'
)

# Configurar eje x
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10
)

# Añadir título y etiquetas
fig.update_layout(
    title='Gráfico de Barras de Diferencias de Ingresos - Gastos',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Tipo de Diferencia',
    width=1000,
    height=600
)

# Mostrar gráfico
fig.show()

In [32]:
import numpy as np
import plotly.graph_objects as go

# Crear listas para las fechas y valores separados por signo
dates_positive = []
values_positive = []
dates_negative = []
values_negative = []

# Listas para excesos
excess_positive_dates = []
excess_positive_values = []
excess_negative_dates = []
excess_negative_values = []

# Calcular diferencias y excesos en cada subintervalo
for i, subinterval in enumerate(subintervals):
    diff_values = subinterval[' Amount'].diff().dropna()
    dates = subinterval['Payment Date'][1:]

    # Percentiles
    threshold_90 = np.percentile(diff_values, 90)
    threshold_10 = np.percentile(diff_values, 15)

    for date, value in zip(dates, diff_values):
        # Clasificar por signo
        if value > 0:
            dates_positive.append(date)
            values_positive.append(value)
        elif value < 0:
            dates_negative.append(date)
            values_negative.append(value)

        # Clasificar por exceso
        if value > threshold_90:
            excess_positive_dates.append(date)
            excess_positive_values.append(value)
        if value < threshold_10:
            excess_negative_dates.append(date)
            excess_negative_values.append(value)

# Crear figura
fig = go.Figure()

# Agregar barras positivas (verdes)
fig.add_trace(go.Bar(
    x=dates_positive,
    y=values_positive,
    name='Diferencias Positivas',
    marker_color='green'
))

# Agregar barras negativas (rojas)
fig.add_trace(go.Bar(
    x=dates_negative,
    y=values_negative,
    name='Diferencias Negativas',
    marker_color='red'
))

# Agregar excesos positivos (círculos rojos)
fig.add_trace(go.Scatter(
    x=excess_positive_dates,
    y=excess_positive_values,
    mode='markers',
    name='Excesos Positivos (> P90)',
    marker=dict(color='darkred', size=10, symbol='circle')
))

# Agregar excesos negativos (triángulos azules)
fig.add_trace(go.Scatter(
    x=excess_negative_dates,
    y=excess_negative_values,
    mode='markers',
    name='Excesos Negativos (< P15)',
    marker=dict(color='blue', size=10, symbol='triangle-down')
))

# Línea horizontal en y=0
fig.add_shape(
    type='line',
    x0=subset['Payment Date'].min(),
    x1=subset['Payment Date'].max(),
    y0=0,
    y1=0,
    line=dict(color='black', width=2, dash='dash'),
    xref='x',
    yref='y'
)

# Eje x con fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10
)

# Layout general
fig.update_layout(
    title='Gráfico de Barras con Excesos Extremos en Ingresos - Gastos',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

fig.show()

In [24]:
import numpy as np

#Vamos a calcular las diferencias y los excesos sobre los percentiles 90 y 10
fig = go.Figure()

#Añadir trazos para las diferencias de cada subintervalo
for i, subinterval in enumerate(subintervals):
    ####Calcular las diferencias consecutivas
    diff_values = subinterval[' Amount'].diff().dropna()
    
    ####Calcular los percentiles 90 y 10 del subintervalo
    threshold_90 = np.percentile(diff_values, 90)
    threshold_10 = np.percentile(diff_values, 15)
    
    ####Identificar los valores que exceden el percentil 90 (excesos positivos)
    excess_positive_values = diff_values[diff_values > threshold_90]
    excess_positive_dates = subinterval['Payment Date'][1:][diff_values > threshold_90]  # Ajustar las fechas
    
    ####Identificar los valores que caen por debajo del percentil 10 (excesos negativos)
    excess_negative_values = diff_values[diff_values < threshold_10]
    excess_negative_dates = subinterval['Payment Date'][1:][diff_values < threshold_10]  # Ajustar las fechas

    ####Añadir el trazo para las diferencias en el subintervalo
    fig.add_trace(go.Scatter(x=subinterval['Payment Date'][1:], y=diff_values,
                             mode='lines', name=f'Periodo de Tiempo {i+1}'))

    ####Añadir el trazo para los excesos sobre el umbral del percentil 90 (picos positivos)
    fig.add_trace(go.Scatter(x=excess_positive_dates, y=excess_positive_values,
                             mode='markers', name=f'Excesos Positivos Periodo de tiempo {i+1}',
                             marker=dict(color='red', size=8, symbol='circle')))

    #####Añadir el trazo para los excesos por debajo del umbral del percentil 10 (picos negativos)
    fig.add_trace(go.Scatter(x=excess_negative_dates, y=excess_negative_values,
                             mode='markers', name=f'Excesos Negativos Peridoo de tiempo negativo {i+1}',
                             marker=dict(color='blue', size=8, symbol='triangle-down')))

####Configurar el formato del eje x para las fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10  #----------Ajusta según la cantidad de etiquetas de fecha que quieras mostrar
)

#####Añadir títulos y etiquetas
fig.update_layout(
    title='Grafico de ingresos y gastos excesivos extremos',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

fig.show()

In [11]:
diff_values = np.array(diff_values)

In [12]:
n_steps = 10

# Crear las secuencias de entrada (X) y las salidas correspondientes (y)
X, y = [], []
for i in range(len(diff_values) - n_steps - 3 + 1):
    X.append(diff_values[i:i + n_steps])
    y.append(diff_values[i + n_steps:i + n_steps + 3])

X = np.array(X)
y = np.array(y)

# Redimensionar X para que sea compatible con la entrada de LSTM: [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense

model = Sequential([
    Input(shape=(n_steps, 1)),
    LSTM(50, activation='relu'),
    Dense(3)
])
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, y, epochs=200, verbose=1, batch_size=4)

Epoch 1/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 140466160.0000
Epoch 2/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 101363552.0000
Epoch 3/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 161594960.0000
Epoch 4/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 137047168.0000
Epoch 5/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 134105416.0000
Epoch 6/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 166963392.0000
Epoch 7/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 147837872.0000
Epoch 8/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 173712064.0000
Epoch 9/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 78114768.0000
Epoch 10/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 162906992.0000
Epoch 11/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 213414768.0000
Epoch 12/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 140349392.0000
Epoch 13/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 155515680.0000
Epoch 14/200
58/58 ━━━━━━━━━━━━━━━━

In [ ]:
import matplotlib.pyplot as plt

# Suponiendo que tenemos los valores reales para comparar
# y_true = [...]

# Crear una lista de fechas para las predicciones
# last_date = subinterval['Payment Date'].iloc[-1]
# prediction_dates = [last_date + pd.Timedelta(days=i) for i in range(1, 4)]

# Graficar
plt.figure(figsize=(10, 6))
plt.plot(subinterval['Payment Date'], diff_values, label='Datos reales')
# plt.plot(prediction_dates, yhat[0], label='Predicciones', marker='o')
plt.title('Predicción de diferencias de montos')
plt.xlabel('Fecha')
plt.ylabel('Diferencia de monto')
plt.legend()
plt.show()